In [7]:
# import time
# import logging
# import sys
# import utils.general as general_util
# from dotenv import load_dotenv
import polars as pl
import utils.parsing_helpers.tournament_parser as tournament_parser
import utils.parsing_helpers.leaderboard_parser as leaderboard_parser
import utils.general as general_util

pl.Config(tbl_width_chars=100)
pl.Config(tbl_rows=100)

In [14]:
tournament_id = "wEuVhT9c"
tournament_json, tournament_api_endpoint = general_util.call_lichess_broadcasts_api(
    tournament_id
)
lichess_tour_df, lichess_rounds_outline_df = (
    tournament_parser.create_broadcast_base_dataframes(tournament_json, tournament_id)
)

tour_slug = lichess_tour_df.select("slug").item()
has_leaderboard = lichess_tour_df.select("leaderboard").item()
lichess_rounds_outline_df = lichess_rounds_outline_df.with_columns(
    tournament_slug=pl.lit(tour_slug)
)

In [9]:
if has_leaderboard:
    leaderboard_json, leaderboard_api_endpoint = (
        general_util.call_lichess_broadcasts_api(tournament_id, leaderboard=True)
    )
    lichess_leaderboard_df = leaderboard_parser.create_broadcast_leaderboard_dataframe(
        leaderboard_json, tournament_id
    )[0]

In [10]:
lichess_rounds_outline_df.with_columns(
    pl.concat_list(["tournament_slug", "slug", "round_id"]).alias("rounds_info")
)

round_id,tournament_id,name,slug,created_at,starts_at,url,finished,tournament_slug,rounds_info
str,str,str,str,i64,i64,str,bool,str,list[str]
"""AjqSsU1w""","""wEuVhT9c""","""Round 1""","""round-1""",1710760290779,1712256300000,"""https://liches…",true,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-1"", ""AjqSsU1w""]"
"""GenKIJ8A""","""wEuVhT9c""","""Round 2""","""round-2""",1711530504000,1712342700000,"""https://liches…",true,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-2"", ""GenKIJ8A""]"
"""xQgaUu2y""","""wEuVhT9c""","""Round 3""","""round-3""",1711530516119,1712429100000,"""https://liches…",true,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-3"", ""xQgaUu2y""]"
"""CPS9dENa""","""wEuVhT9c""","""Round 4""","""round-4""",1711530526339,1712515500000,"""https://liches…",true,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-4"", ""CPS9dENa""]"
"""MDiLWQ5M""","""wEuVhT9c""","""Round 5""","""round-5""",1711530555192,1712688300000,"""https://liches…",true,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-5"", ""MDiLWQ5M""]"
"""kPqEUBZJ""","""wEuVhT9c""","""Round 6""","""round-6""",1711530566963,1712774700000,"""https://liches…",true,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-6"", ""kPqEUBZJ""]"
"""X9LGGQoT""","""wEuVhT9c""","""Round 7""","""round-7""",1711530578504,1712861100000,"""https://liches…",true,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-7"", ""X9LGGQoT""]"
"""nUycmG6L""","""wEuVhT9c""","""Round 8""","""round-8""",1711530594895,1713033900000,"""https://liches…",true,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-8"", ""nUycmG6L""]"
"""A7SWsX0x""","""wEuVhT9c""","""Round 9""","""round-9""",1711530611752,1713120300000,"""https://liches…",null,"""fide-candidate…","[""fide-candidates-2024--open"", ""round-9"", ""A7SWsX0x""]"


In [11]:
rounds_json, rounds_api_endpoint = general_util.call_lichess_broadcasts_api(
    tournament_id,
    round_info=["fide-candidates-2024--open", "round-5", "MDiLWQ5M"],
    # round_info=["fide-candidates-2024--open", "round-8", "nUycmG6L"]
    # round_info=["fide-candidates-2024--open", "round-14", "S4zisI6M"]
)
# lichess_rounds_detail_df = tournament_helper.create_broadcast_leaderboard_dataframe(
#     rounds_json, tournament_id
# )[0]

In [12]:
rounds_json

{'round': {'id': 'MDiLWQ5M',
  'name': 'Round 5',
  'slug': 'round-5',
  'createdAt': 1711530555192,
  'finished': True,
  'startsAt': 1712688300000,
  'url': 'https://lichess.org/broadcast/fide-candidates-2024--open/round-5/MDiLWQ5M',
  'delay': 0},
 'tour': {'id': 'wEuVhT9c',
  'name': 'FIDE Candidates 2024 | Open',
  'slug': 'fide-candidates-2024--open',
  'description': 'April 4th - 22nd | 8-player double round-robin | Classical time control',
  'createdAt': 1710759962579,
  'tier': 5,
  'image': 'https://image.lichess1.org/display?h=400&op=thumbnail&path=loepare:relay:wEuVhT9c:UzQX0Wlo.jpg&w=800&sig=a02fbde968e0095d83e48a6055c16254ee5aa331'},
 'study': {'writeable': False},
 'games': [{'id': 'MI1pR0AA',
   'name': 'Praggnanandhaa R - Nepomniachtchi, Ian',
   'fen': '5k2/1R6/1pr5/6p1/p5P1/6K1/P4P2/8 w - - 0 45',
   'players': [{'name': 'Praggnanandhaa R',
     'title': 'GM',
     'rating': 2747,
     'clock': 258200,
     'fed': 'IND'},
    {'name': 'Nepomniachtchi, Ian',
     'tit

In [13]:
lichess_rounds_outline_df

round_id,tournament_id,name,slug,created_at,starts_at,url,finished,tournament_slug
str,str,str,str,i64,i64,str,bool,str
"""AjqSsU1w""","""wEuVhT9c""","""Round 1""","""round-1""",1710760290779,1712256300000,"""https://liches…",true,"""fide-candidate…"
"""GenKIJ8A""","""wEuVhT9c""","""Round 2""","""round-2""",1711530504000,1712342700000,"""https://liches…",true,"""fide-candidate…"
"""xQgaUu2y""","""wEuVhT9c""","""Round 3""","""round-3""",1711530516119,1712429100000,"""https://liches…",true,"""fide-candidate…"
"""CPS9dENa""","""wEuVhT9c""","""Round 4""","""round-4""",1711530526339,1712515500000,"""https://liches…",true,"""fide-candidate…"
"""MDiLWQ5M""","""wEuVhT9c""","""Round 5""","""round-5""",1711530555192,1712688300000,"""https://liches…",true,"""fide-candidate…"
"""kPqEUBZJ""","""wEuVhT9c""","""Round 6""","""round-6""",1711530566963,1712774700000,"""https://liches…",true,"""fide-candidate…"
"""X9LGGQoT""","""wEuVhT9c""","""Round 7""","""round-7""",1711530578504,1712861100000,"""https://liches…",true,"""fide-candidate…"
"""nUycmG6L""","""wEuVhT9c""","""Round 8""","""round-8""",1711530594895,1713033900000,"""https://liches…",true,"""fide-candidate…"
"""A7SWsX0x""","""wEuVhT9c""","""Round 9""","""round-9""",1711530611752,1713120300000,"""https://liches…",null,"""fide-candidate…"


In [15]:
tour_slug = lichess_tour_df.select("slug").item()

'fide-candidates-2024--open'

In [20]:
lichess_rounds_outline_df

round_id,tournament_id,name,slug,created_at,starts_at,url,finished
str,str,str,str,i64,i64,str,bool
"""AjqSsU1w""","""wEuVhT9c""","""Round 1""","""round-1""",1710760290779,1712256300000,"""https://liches…",true
"""GenKIJ8A""","""wEuVhT9c""","""Round 2""","""round-2""",1711530504000,1712342700000,"""https://liches…",true
"""xQgaUu2y""","""wEuVhT9c""","""Round 3""","""round-3""",1711530516119,1712429100000,"""https://liches…",true
"""CPS9dENa""","""wEuVhT9c""","""Round 4""","""round-4""",1711530526339,1712515500000,"""https://liches…",true
"""MDiLWQ5M""","""wEuVhT9c""","""Round 5""","""round-5""",1711530555192,1712688300000,"""https://liches…",true
…,…,…,…,…,…,…,…
"""46ohJ8Qt""","""wEuVhT9c""","""Round 11""","""round-11""",1711530640577,1713379500000,"""https://liches…",null
"""eJghIBZe""","""wEuVhT9c""","""Round 12""","""round-12""",1711530651369,1713465900000,"""https://liches…",null
"""Dc0DQMaQ""","""wEuVhT9c""","""Round 13""","""round-13""",1711530662643,1713638700000,"""https://liches…",null


In [21]:
lichess_leaderboard_df

fide_id,tournament_id,name,score,number_of_rounds_played,rating,title,federation
i64,str,str,f64,i64,i64,str,str
4168119,"""wEuVhT9c""","""Nepomniachtchi…",4.5,7,2758,"""GM""","""RUS"""
2020009,"""wEuVhT9c""","""Caruana, Fabia…",4.0,7,2803,"""GM""","""USA"""
25059530,"""wEuVhT9c""","""Praggnanandhaa…",4.0,7,2747,"""GM""","""IND"""
46616543,"""wEuVhT9c""","""Gukesh D""",4.0,7,2743,"""GM""","""IND"""
2016192,"""wEuVhT9c""","""Nakamura, Hika…",3.5,7,2789,"""GM""","""USA"""
5029465,"""wEuVhT9c""","""Vidit, Santosh…",3.5,7,2727,"""GM""","""IND"""
12573981,"""wEuVhT9c""","""Firouzja, Alir…",2.5,7,2760,"""GM""","""FRA"""
13402960,"""wEuVhT9c""","""Abasov, Nijat""",2.0,7,2632,"""GM""","""AZE"""


In [2]:
lichess_rounds_detail_df

NameError: name 'lichess_rounds_detail_df' is not defined

In [13]:
# tournament_helper.call_lichess_broadcasts_api(tournament_id, round_info=["japan-youth-chess-championship-2024", "round-1", "i0QWBTTi"])

In [50]:
import json

with open(f"{tournament_id}_8.json", "w") as f:
    json.dump(rounds_json, f)